#Unsupervised Classification

Unsupervised classification หรือ ในภาษาไทย เรียกว่า "การจำแนกประเภทแบบไม่มีผู้ดูแล.​ใน Google Earth Engine (GEE) เป็นกระบวนการเรียนรู้ของเครื่องที่จัดกลุ่มพิกเซลเป็นกลุ่มตามคุณสมบัติทางสเปกตรัมโดยไม่ต้องทำการ labeling โดยทั่วไปวิธีนี้ใช้ในการระบุรูปแบบตามธรรมชาติและจัดหมวดหมู่ประเภทสิ่งปกคลุมดินในภาพถ่ายดาวเทียม

กระบวนการเริ่มต้นด้วยการเลือกภาพถ่ายดาวเทียมและแถบสเปกตรัม (Bands) ที่เหมาะสมสำหรับการวิเคราะห์ ขั้นตอนการประมวลผลล่วงหน้า เช่น การมาสก์คลาวด์และการทำให้ normalization เพื่อเตรียมข้อมูล

สำหรับการจำแนกประเภท ต้องกำหนดจำนวนคลัสเตอร์ (k) ซึ่งอาจส่งผลต่อผลลัพธ์อย่างมีนัยสำคัญ

การจัดหมวดหมู่จะดำเนินการโดยใช้อัลกอริธึม เช่น การจัดกลุ่มแบบ k-means ซึ่งจัดพิกเซลออกเป็น k กลุ่มตามความคล้ายคลึงกันทางสเปกตรัม หลังจากการจำแนกประเภท แผนที่ผลลัพธ์จะแสดงแต่ละพิกเซลที่มีป้ายกำกับตามคลัสเตอร์ ซึ่งอาจต้องมีการตีความหรือปรับแต่งเพิ่มเติมตามความรู้ของผู้เชี่ยวชาญ

In [ ]:
#เผื่อใครที่ยังไม่ได้ Install ให้รัน cell นี้ก่อน ถ้า Install แล้วข้ามไปได้เลย
!pip install geemap
!pip install earthengine-api

In [ ]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-wutnongkan') # อย่าลืมเปลี่ยนเป็นProject

In [ ]:
# roi = ee.FeatureCollection('projects/ee-pythoncolab/assets/BMR_test') # อย่าลืมเปลี่ยนเป็นProject
roi = ee.FeatureCollection("projects/ee-wutnongkan/assets/Tambon_dopa") \
        .filter(ee.Filter.eq('tambon_t', 'ต.เรือง'))
l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
start = '2000-01-01'
end = '2000-12-31'

def cloud_mask(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 1).eq(0) \
        .And(qa.bitwiseAnd(1 << 3).eq(0)) \
        .And(qa.bitwiseAnd(1 << 2).eq(0)) \
        .And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask) \
        .select(['SR_B.*'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) \
        .multiply(0.0000275).add(-0.2)

image = l5.filterDate(start, end) \
    .filterBounds(roi) \
    .map(cloud_mask) \
    .median() \
    .clip(roi)

In [ ]:
training = image.sample(region=roi, scale=20, numPixels=50)
num_clusters = 10
clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(training)
result = image.cluster(clusterer)

Map = geemap.Map()
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map.centerObject(roi, 13)
Map